In [1]:
# Load contractions model
from pycontractions import Contractions

cont = Contractions(api_key="glove-twitter-100")
cont.load_models()

### Parameters

In [36]:
file = 'Data/c3.csv'
ngram = (1,3)
mindf = 2
maxdf = 0.8

### Load

In [37]:
import pandas as pd

df = pd.read_csv(file,sep=",")
display(df.head())

,id,text,author,date,keywords,organization,rating
0,http://data.gesis.org/claimskg/creative_work/c...,Says Obama adviser David Plouffe took 'large p...,Secure America Now,09/12/2019,"candidate biography,foreign policy,message mac...",politifact,MIXTURE
1,http://data.gesis.org/claimskg/creative_work/5...,"'For his first interview as president, Barack ...",Secure America Now,07/04/2019,"foreign policy,message machine 2012",politifact,MIXTURE
2,http://data.gesis.org/claimskg/creative_work/0...,'A few months ago when you were asked what's t...,Barack Obama,03/25/2019,"corrections and updates,debates,foreign policy",politifact,MIXTURE
3,http://data.gesis.org/claimskg/creative_work/0...,Russia and China are 'doing naval exercises to...,Donald Trump,03/25/2019,"china,foreign policy,terrorism",politifact,MIXTURE
4,http://data.gesis.org/claimskg/creative_work/0...,'The president said he’s going to bring in 250...,Sean Hannity,03/25/2019,"foreign policy,immigration,terrorism",politifact,FALSE


In [38]:
df5 = df.copy().sort_values(by="author")
df6 = df5.loc[(df5['organization'] != 'snopes')]
authors = df6["author"].unique()
#display(df6)

print(df6.shape)
len(authors)

(1377, 7)


498

In [39]:
df1 = df.copy()
df1 = df1.loc[df1['rating'] != 'OTHER']
display(df1.head())

,id,text,author,date,keywords,organization,rating
0,http://data.gesis.org/claimskg/creative_work/c...,Says Obama adviser David Plouffe took 'large p...,Secure America Now,09/12/2019,"candidate biography,foreign policy,message mac...",politifact,MIXTURE
1,http://data.gesis.org/claimskg/creative_work/5...,"'For his first interview as president, Barack ...",Secure America Now,07/04/2019,"foreign policy,message machine 2012",politifact,MIXTURE
2,http://data.gesis.org/claimskg/creative_work/0...,'A few months ago when you were asked what's t...,Barack Obama,03/25/2019,"corrections and updates,debates,foreign policy",politifact,MIXTURE
3,http://data.gesis.org/claimskg/creative_work/0...,Russia and China are 'doing naval exercises to...,Donald Trump,03/25/2019,"china,foreign policy,terrorism",politifact,MIXTURE
4,http://data.gesis.org/claimskg/creative_work/0...,'The president said he’s going to bring in 250...,Sean Hannity,03/25/2019,"foreign policy,immigration,terrorism",politifact,FALSE


In [40]:
t = df1.loc[(df1['rating'] == 'TRUE')]['id'].count()
f = df1.loc[(df1['rating'] == 'FALSE')]['id'].count()
m = df1.loc[(df1['rating'] == 'MIXTURE')]['id'].count()

print('True claims : %d' % t)
print('False claims : %d' % f)
print('Mixture claims : %d' % m)
print('TF claims : %d' % (t+f))

True claims : 209
False claims : 438
Mixture claims : 763
TF claims : 647


### Preprocess

In [41]:
import re

# def preprocess(sentence):
#     sentence = sentence.lower().strip()
#     translator = str.maketrans('’', "'", '')
#     sentence = sentence.translate(translator)
#     sentence = re.sub("u\.s\.","united states",sentence)
#     sentence = list(cont.expand_texts([sentence],precise=True))[0]
#     sentence = re.sub("[^a-zA-Z0-9_.’,]|(?<!\d)\.(?!\d)|(?<!\w)-(?!\w)|(?<!\d)\,(?!\d)",' ',sentence)
#     sentence = re.sub(",",'',sentence)
#     sentence = re.sub('\s+', ' ', sentence).strip()
#     sentence = re.sub(" s ",' ',sentence)
#     if sentence[0:5] == 'says ':
#         sentence = sentence[5:]
#     return sentence

def preprocess(sentence):
    sentence = sentence.lower().strip()
    translator = str.maketrans('’', "'", '')
    sentence = sentence.translate(translator)
    sentence = re.sub("u\.s\.","united states",sentence)
    sentence = list(cont.expand_texts([sentence],precise=True))[0]
    sentence = re.sub("[^a-zA-Z0-9_.’,]|(?<!\d)\.(?!\d)|(?<!\w)-(?!\w)|(?<!\d)\,(?!\d)",' ',sentence)
    sentence = re.sub(",",'',sentence)
    sentence = re.sub("\.",'',sentence)
    sentence = re.sub(" a ",' ',sentence)
    sentence = re.sub('\s+', ' ', sentence).strip()
    sentence = re.sub(" s ",' ',sentence)
    if sentence[0:5] == 'says ':
        sentence = sentence[5:]
    sentence = ' '.join([w for w in sentence.split() if len(w)>1])
    return sentence

In [42]:
for index in df1.index:
    df1.at[index,'text'] = preprocess(df1.at[index,'text'])
    rating = df1.at[index,'rating']
    if rating == 'FALSE':
        df1.at[index,'rating'] = 0
    if rating == 'TRUE':
        df1.at[index,'rating'] = 1
    if rating == 'MIXTURE':
        df1.at[index,'rating'] = 2

In [43]:
for index in df1.index:
    if index < 20:
        print("%s\n" % df1.at[index,'text'])

obama adviser david plouffe took large piles of cash from joint venture partner of company owned by iran revolutionary guard

for his first interview as president barack obama chose arab tv for an apology

few months ago when you we are asked what has the biggest geopolitical threat facing america you said russia

russia and china are doing naval exercises together someplace

the president said he is going to bring in 250000 syrian and iraqi refugees into this country

donald trump publicly invited putin to hack into americans emails

al franken said almost exactly what trump said about john mccain military record

now we have isis coming over the united states mexico border

the pass rate for high school level test to join the military is 35 to 40 percent

it is more difficult to obtain costco membership than it is to get nyc municipal id

donald trump has outlined policies that read like kremlin wish list

led the fight to require the coast guard to buy their engines from us not fore

### Create additional dataframes

In [44]:
#df1 = true/false/mixed
#df2 = true/false
#df3 = true-false/mixed

In [45]:
df2 = df1.copy()
df2 = df2.loc[(df2['rating'] == 0) | (df2['rating'] == 1)]
df2.head()

,id,text,author,date,keywords,organization,rating
4,http://data.gesis.org/claimskg/creative_work/0...,the president said he is going to bring in 250...,Sean Hannity,03/25/2019,"foreign policy,immigration,terrorism",politifact,0
7,http://data.gesis.org/claimskg/creative_work/0...,now we have isis coming over the united states...,Jeff DeWit,03/25/2019,terrorism,politifact,0
8,http://data.gesis.org/claimskg/creative_work/0...,the pass rate for high school level test to jo...,Jeb Bush,03/25/2019,"education,military",politifact,0
9,http://data.gesis.org/claimskg/creative_work/0...,it is more difficult to obtain costco membersh...,"Ron Castorina, Jr.",03/25/2019,"immigration,terrorism",politifact,0
12,http://data.gesis.org/claimskg/creative_work/0...,ron johnson helped companies ship jobs overseas,Russ Feingold,03/25/2019,"foreign policy,jobs,trade",politifact,0


In [46]:
df3 = df1.copy()
for index in df3.index:
    rating = df3.at[index,'rating']
    if rating == 1:
        df3.at[index,'rating'] = 0
    if rating == 2:
        df3.at[index,'rating'] = 1
        
df3.head()

,id,text,author,date,keywords,organization,rating
0,http://data.gesis.org/claimskg/creative_work/c...,obama adviser david plouffe took large piles o...,Secure America Now,09/12/2019,"candidate biography,foreign policy,message mac...",politifact,1
1,http://data.gesis.org/claimskg/creative_work/5...,for his first interview as president barack ob...,Secure America Now,07/04/2019,"foreign policy,message machine 2012",politifact,1
2,http://data.gesis.org/claimskg/creative_work/0...,few months ago when you we are asked what has ...,Barack Obama,03/25/2019,"corrections and updates,debates,foreign policy",politifact,1
3,http://data.gesis.org/claimskg/creative_work/0...,russia and china are doing naval exercises tog...,Donald Trump,03/25/2019,"china,foreign policy,terrorism",politifact,1
4,http://data.gesis.org/claimskg/creative_work/0...,the president said he is going to bring in 250...,Sean Hannity,03/25/2019,"foreign policy,immigration,terrorism",politifact,0


In [47]:
dfs = dict()
dfs['3 Classes'] = df1
dfs['2 Classes'] = df2
dfs['2 Mixed Classes'] = df3

### Vectors

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize(df):
    vectorizer = TfidfVectorizer(ngram_range=ngram,min_df=mindf,max_df=maxdf)
    vectors = vectorizer.fit_transform(df['text']).toarray()
    #print(vectorizer.get_feature_names())
    return vectors

### Classification

In [49]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

def classify(df, key):
    X = vectorize(df)
    y = df["rating"]
    y = y.astype('int')
    clf = LinearSVC(random_state=0, tol=1e-5)
    k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)
    score = cross_val_score(clf, X, y, cv=k_fold, scoring='accuracy')
    print('%s - Mean accuracy: %f, Deviation: %f' % (key,score.mean(),score.std()))

In [50]:
for key in dfs:
    classify(dfs[key], key)

3 Classes - Mean accuracy: 0.533456, Deviation: 0.053546
2 Classes - Mean accuracy: 0.706087, Deviation: 0.064653
2 Mixed Classes - Mean accuracy: 0.576719, Deviation: 0.034614


### Feature Selection

In [51]:
from sklearn.feature_selection import mutual_info_classif

def get_top_features(df):
    vectorizer = TfidfVectorizer(ngram_range=ngram,min_df=mindf,max_df=maxdf)
    vectors = vectorizer.fit_transform(df['text']).toarray()
    feature_scores = mutual_info_classif(vectors, df['rating'].astype('int'))
    
    for score, fname in sorted(zip(feature_scores, vectorizer.get_feature_names()), reverse=True)[:30]:
        print(fname, score)

In [52]:
# for key in dfs:
#     print("********** %s **********" % key)
#     get_top_features(dfs[key])
#     print()